In [68]:
import pandas as pd
import numpy as np
import os
from collections import deque

pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)

# Day 3 Gear Ratios
You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:

467..114..\
...*......\
..35..633.\
......#...\
617*......\
.....+.58.\
..592.....\
......755.\
...$.*....\
.664.598..\

In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

In [69]:
with open('input.txt', 'r') as file:
    input = file.read()

# input = '''467..114..
# ...*......
# ..35..633.
# ......#...
# 617*......
# .....+.58.
# ..592.....
# ......755.
# ...$.*....
# .664.598..'''

DF = pd.DataFrame(pd.Series(input.split('\n')).apply(list).to_list()).dropna()

DF = pd.concat([pd.Series(['.']*len(DF), name='StartDot'), DF], axis=1)
DF = pd.concat([DF, pd.Series(['.']*len(DF), name='EndDot')], axis=1)

# Add a row with '.' to the first and last positions
new_row = pd.DataFrame([['.']*len(DF.columns)], columns=DF.columns)
DF = pd.concat([new_row, DF]).reset_index(drop=True)
DF = pd.concat([DF, new_row]).reset_index(drop=True)
DF.columns = range(len(DF.columns))

PART_NUMBERS = {}

In [70]:
DF

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141
0,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
1,.,.,6,7,9,.,.,.,.,.,6,6,2,.,.,.,.,7,1,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,8,0,5,.,.,.,.,.,.,.,.,.,.,8,6,2,.,6,8,0,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,6,8,7,.,.,.,.,.
2,.,.,.,.,.,.,.,.,.,.,.,.,.,*,.,.,.,.,-,.,.,8,1,1,.,.,.,.,.,.,.,.,.,.,8,4,6,.,.,8,5,5,.,.,.,.,.,.,*,.,.,.,.,.,.,.,.,.,.,.,.,.,*,.,.,$,.,.,.,.,.,.,.,.,2,3,0,.,9,2,@,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,=,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,9,2,.,.,.,.,.,.,.,.,.
3,.,.,.,.,.,.,.,.,.,.,.,3,6,0,.,.,.,.,.,.,.,.,.,.,#,.,.,.,.,6,6,4,.,.,.,.,.,=,.,*,.,.,.,8,8,1,.,.,.,6,7,7,.,.,.,9,3,4,.,7,8,0,.,.,.,.,.,.,.,4,2,6,.,*,.,.,.,.,.,.,.,.,.,.,8,.,.,.,.,.,.,6,5,4,.,.,.,.,.,*,9,5,9,.,.,.,.,.,5,3,9,.,.,.,.,.,.,.,.,.,.,2,1,.,.,.,.,.,.,.,.,.,*,.,.,.,.,.,.,.,.,.
4,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,+,.,.,.,.,.,.,.,.,.,*,.,.,.,.,.,.,3,7,9,.,.,*,.,.,.,.,.,.,.,.,.,*,.,.,.,.,.,.,.,.,.,=,.,.,.,.,.,.,.,.,.,9,6,9,.,.,.,.,.,.,.,.,*,.,.,.,.,.,.,.,.,*,.,9,7,6,.,.,.,.,.,.,.,.,.,.,.,.,.,.,8,7,2,.,.,.,.,*,.,.,.,.,.,.,/,.,.,.,.,5,7,9,.,.,.,.,.,.,.
5,.,.,.,.,.,.,.,.,5,6,6,.,.,.,.,.,.,6,5,2,.,.,.,8,0,9,.,.,.,.,4,8,2,.,3,9,4,.,.,.,.,.,.,4,9,2,.,.,3,0,3,.,6,5,0,.,.,.,.,/,.,.,.,3,8,.,.,.,.,%,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,1,0,6,.,.,.,3,8,5,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,#,.,.,.,.,.,7,9,3,.,.,4,8,4,.,8,6,5,.,.,.,.,.,.,.,.,.,.,.,.
6,.,.,.,.,.,.,.,*,.,.,.,.,.,.,.,.,.,.,.,*,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,*,.,.,3,4,7,.,.,.,.,.,.,.,*,.,.,.,.,.,.,.,.,.,2,2,0,.,.,.,.,.,3,4,9,.,.,.,6,9,1,.,.,.,3,9,2,*,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,1,8,.,.,7,9,7,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,+,.,.,.,.,.,.,.,.,.,.
7,.,.,.,.,.,.,1,1,.,.,.,.,.,.,8,9,0,.,.,.,.,.,.,.,.,8,7,0,.,.,.,.,.,.,.,1,5,6,.,.,.,.,.,.,.,.,.,.,.,.,7,3,3,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,/,.,.,.,.,.,.,.,.,.,.,9,2,1,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,*,.,.,.,.,.,.,*,.,.,.,.,.,.,.,.,.,.,.,.,.,2,0,3,.,.,.,.,.,.,.,.,.,.,.
8,.,.,.,.,.,.,.,.,.,.,.,.,.,.,#,.,.,.,2,3,8,.,.,.,.,.,*,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,1,8,8,.,.,.,.,.,.,2,9,4,.,/,5,8,.,.,.,.,.,.,.,.,4,0,8,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,6,7,7,.,.,.,.,.,.,.,7,7,8,.,.,.,.,1,0,4,.,9,0,3,.,.,.,.,.,.,.,.,.,.,.,.,%,.,.,.,.,.,.,.,.,.
9,.,.,4,1,1,+,.,.,.,8,2,8,.,.,.,.,.,.,.,*,.,.,.,.,.,7,0,6,.,.,.,.,.,2,4,9,.,.,=,.,.,.,.,.,6,3,8,.,.,.,8,4,8,*,.,.,.,.,.,.,.,.,.,.,.,%,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,*,9,4,8,.,.,.,.,.,.,.,.,.,.,.,.,*,.,.,.,.,.,+,.,.,.,.,.,.,+,.,.,.,.,.,.,.,.,.,.,.,9,1,7,.,.,.,.,.,.,.,8,1,7,.,.,.,3,4,6,.


In [71]:
class Tile:
    def __init__(self, i, j):
        self.i = i
        self.j = j

    def get(self):
        return DF.iloc[self.i, self.j]
    
    def check_number(self):
        if self.get().isdigit():
            return True
        return False
    
## create a class to manage checking surroundings
class Map:
    def __init__(self, i, j):
        self.c = Tile(i, j)
        self.r = Tile(i, j+1)
        self.l = Tile(i, j-1)
        self.u = Tile(i-1, j)
        self.d = Tile(i+1, j)
        self.ur = Tile(i-1, j+1)
        self.ul = Tile(i-1, j-1)
        self.dr = Tile(i+1, j+1)
        self.dl = Tile(i+1, j-1)
    

def add_part_number(m: Map):
    part_dqueue = deque([m.c.get()])
    temp_m = Map(m.l.i, m.l.j)

    # check left tiles
    while temp_m.c.check_number():
        part_dqueue.appendleft(temp_m.c.get())
        temp_m = Map(temp_m.l.i, temp_m.l.j)
    
    temp_m = Map(m.r.i, m.r.j)

    # check right tiles
    while temp_m.c.check_number():
        part_dqueue.append(temp_m.c.get())
        temp_m = Map(temp_m.r.i, temp_m.r.j)

    part_number = int(''.join(map(str, part_dqueue)))

    # check if part is already in dict
    if (temp_m.l.i, temp_m.l.j) not in PART_NUMBERS.keys():
        print(f'Found Part Number -> ({temp_m.l.i}, {temp_m.l.j}): {part_number}')
        PART_NUMBERS[(temp_m.l.i, temp_m.l.j)] = part_number

    
def check_around_map(m: Map):
    try:
        if m.r.check_number():
            add_part_number(Map(m.r.i, m.r.j))
    except IndexError:
        pass

    try:
        if m.l.check_number():
            add_part_number(Map(m.l.i, m.l.j))
    except IndexError:
        pass
    
    try:
        if m.u.check_number():
            add_part_number(Map(m.u.i, m.u.j))
    except IndexError:
        pass

    try:
        if m.d.check_number():
            add_part_number(Map(m.d.i, m.d.j))
    except IndexError:
        pass
    
    try:
        if m.ur.check_number():
            add_part_number(Map(m.ur.i, m.ur.j))
    except IndexError:
        pass

    try:
        if m.ul.check_number():
            add_part_number(Map(m.ul.i, m.ul.j))
    except IndexError:
        pass
    
    try:
        if m.dl.check_number():
            add_part_number(Map(m.dl.i, m.dl.j))
    except IndexError:
        pass

    try:
        if m.dr.check_number():
            add_part_number(Map(m.dr.i, m.dr.j))
    except IndexError:
        pass

def check_current_map(m: Map):
    if m.c.get() != '.' and not m.c.get().isdigit():
        return True
    return False



In [72]:
for i in range(len(DF)):
    for j in range(len(DF.columns)):
        current_map = Map(i, j)
        
        # check if we are at a special character
        if check_current_map(current_map):
            check_around_map(current_map)

Found Part Number -> (3, 13): 360
Found Part Number -> (1, 12): 662
Found Part Number -> (1, 18): 71
Found Part Number -> (1, 49): 805
Found Part Number -> (3, 51): 677
Found Part Number -> (1, 62): 862
Found Part Number -> (3, 61): 780
Found Part Number -> (1, 66): 680
Found Part Number -> (2, 79): 92
Found Part Number -> (3, 110): 539
Found Part Number -> (2, 23): 811
Found Part Number -> (2, 36): 846
Found Part Number -> (2, 41): 855
Found Part Number -> (4, 41): 379
Found Part Number -> (2, 76): 230
Found Part Number -> (4, 76): 969
Found Part Number -> (3, 102): 959
Found Part Number -> (4, 98): 976
Found Part Number -> (2, 132): 92
Found Part Number -> (4, 134): 579
Found Part Number -> (5, 25): 809
Found Part Number -> (5, 32): 482
Found Part Number -> (3, 31): 664
Found Part Number -> (3, 45): 881
Found Part Number -> (5, 45): 492
Found Part Number -> (5, 54): 650
Found Part Number -> (3, 57): 934
Found Part Number -> (5, 64): 38
Found Part Number -> (5, 87): 106
Found Part Num

In [73]:
sum(PART_NUMBERS.values())

528799